In [1]:
"""
"""
#import libraries

import pandas as pd # dataframe library
import numpy as np #mathematical library
import datetime #date library
from dateutil.relativedelta import relativedelta #subtract periods to a date
import sys #system exit
import time

#import our sql script to connecto to engine and return dataframe. In this case, 
#the %run is used to "import" our sql connection notebook

%run ..\sql\connect_sql_server.ipynb
#from ipynb.fs.full.connect_sql_server import querySQL --> this is another option to import another notebook when in the same folder

#import warnings library to then avoid the warnings given by jupyter
import warnings
warnings.filterwarnings('ignore')



In [5]:
country = 'Republica Dominicana' #state the country
start_date_history = datetime.datetime(2013, 1, 1) #the start date for our model to analyze
end_date_history = datetime.datetime(2023, 1 , 24) #the end date for our model to analyze
df_history = querySQL(  "{CALL SCAC_AP20_BaseDesagregacionV2 (?,?,?)}", (country, start_date_history.strftime("%Y-%m-%d"), end_date_history.strftime("%Y-%m-%d") ) )


In [6]:
year_disaggregate = 2023 # year to disaggregate 
month_disaggregate = 2
volume = 0 #only for abs_type = 1
inactivate_temp = [] #place inactive factories if there are any
abs_type = -1
"""
PARAMETROS:
absorcionEstadistica = 1  -> get general volume
absorcionEstadistica = 0  -> get volume by city
absorcionEstadistica = -1 -> get volume by factory

"""
#get weeks corresponding to the desired month
calendar_sql = querySQL( "select * from SCAC_AT3_DiasHabilesFuente where pais = ? and año = ? and mes = ? order by [Fecha de entrega]", (country,year_disaggregate,month_disaggregate) )
#get active factories to filter out inactive
active_factories = querySQL( "select Centro, [Planta Unica] as Planta, [Desc Cluster] as Cluster, Ciudad_Cluster as Ciudad  from SCAC_AT1_NombreCluster where pais = ? and activo = 1 order by Centro", (country) )
#execute dissagregation
df_pivot_stational_factor = get_stational_factors(df_history,country,year_disaggregate,month_disaggregate,inactivate_temp,calendar_sql,volume,abs_type,active_factories)
send_excel(df_pivot_stational_factor,country)

no weeks missing
exitoso
